<a href="https://colab.research.google.com/github/ryandoyle5401/CMPSC472HW14/blob/main/HW14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%writefile test.c

#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>
#include <fcntl.h>
#include <sys/stat.h>
#include <sys/types.h>
#include <string.h>
#include <pthread.h>

#define BUFFER_SIZE 1024


//Note: the create_file, read_file are from the CMPSC472_11 colab.

// Used for creating and writing to a file. Probably create a function for writing to a file.
void create_file(const char *filename) {
    int fd = open(filename, O_CREAT | O_RDWR | O_TRUNC, S_IRUSR | S_IWUSR); /* The S_IRUSR and S_IWUSR flags are used to set the permissions for the newly created file.*/
    if (fd == -1) {
        perror("open");
        exit(EXIT_FAILURE);
    }

    // Write some data to the file
    const char *data = "Hello, ext4!";
    if (write(fd, "Writing to file\n", 17) == -1) {
        perror("write");
        close(fd);
        exit(EXIT_FAILURE);
    }

    printf("File '%s' created and written.\n", filename);

    // Close the file descriptor
    close(fd);
}

// Used for reading from a file.
void read_file(const char *filename) {
    int fd = open(filename, O_RDONLY);
    if (fd == -1) {
        perror("open");
        exit(EXIT_FAILURE);
    }

    char buffer[BUFFER_SIZE];
    ssize_t bytes_read;

    // Read data from the file
    bytes_read = read(fd, buffer, BUFFER_SIZE-1);
    if (bytes_read == -1) {
        perror("read");
        close(fd);
        exit(EXIT_FAILURE);
    }

    // Null-terminate the buffer to print as string
    buffer[bytes_read] = '\0';

    printf("Read from file '%s': %s\n", filename, buffer);

    // Close the file descriptor
    close(fd);
}

int main() {
    printf("Hello World\n");
    const char *filename = "ext4_testfile.txt";

    // Create a file and write to it
    create_file(filename);

    // Read from the file
    read_file(filename);


    return 0;
}


Overwriting test.c


In [ ]:
%%shell
gcc test.c -o test
./test

Hello World
File 'ext4_testfile.txt' created and written.
Read from file 'ext4_testfile.txt': Writing to file



In [ ]:
%%writefile test.c
#include <stdio.h>
#include <unistd.h>
#include <fcntl.h>
#include <string.h>
#include <stdlib.h>

int main() {
    int fd = open("example.txt", O_RDWR | O_CREAT, S_IRUSR | S_IWUSR);
    if (fd == -1) {
        perror("Failed to open file");
        exit(1);
    }






// Note: This code is kind of the answer to the hw. Just need to modify to include semaphores





    struct flock fl;
    memset(&fl, 0, sizeof(fl));

    fl.l_type = F_WRLCK;  // Exclusive write lock
    fl.l_whence = SEEK_SET;
    fl.l_start = 0;       // Start of the file
    fl.l_len = 0;         // Lock entire file
    fl.l_pid = getpid();

    // Parent process applies the lock
    if (fcntl(fd, F_SETLK, &fl) == -1) {
        perror("Failed to set lock by parent");
        close(fd);
        exit(1);
    }
    printf("Parent: File locked.\n");

    pid_t child_pid = fork();

    if (child_pid == 0) { // Child process
        // Check the lock status using the same `fl` struct
        if (fcntl(fd ,F_GETLK , &fl) == 0 && fl.l_type != F_UNLCK) { //use F_GETLK, fl_type, and F_UNLCK to fill out the code
            printf("Child: File is locked by another process (PID: %d), cannot write.\n", fl.l_pid);
        } else {
            printf("Child: Lock is free, writing to the file.\n");
            write(fd, "Child process writes to the file.\n", 33);
        }
        sleep(1); // Delay to let child try writing
        // Check the lock status using the same `fl` struct
        if (fcntl(fd, F_GETLK, &fl) == 0 && fl.l_type != F_UNLCK) {
            printf("Child: File is locked by another process (PID: %d), cannot write.\n", fl.l_pid);
        } else {
            printf("Child: Lock is free, writing to the file.\n");
            write(fd, "Child process writes to the file.\n", 33);
        }

    } else { // Parent process
        sleep(0.5); // Delay to let child try writing
        printf("Parent: Writing to the file.\n");
        write(fd, "Parent process writes to the file.\n", 34);
       // Release the lock
        fl.l_type = F_UNLCK;
        fcntl(fd, F_SETLK, &fl);
    }

    close(fd);
    return 0;
}